In [ ]:
pip install botorch

In [4]:
import pandas as pd
import torch

train_data = pd.read_excel('data_sample_multiple_components.xlsx')
#print(train_data)
train_x = train_data.iloc[:,[0,1]].values
#print(train_X)
train_x = torch.tensor(train_x, dtype=torch.float32)
print(train_x)

train_obj = train_data.iloc[:,2:].values
#print(train_Y)
train_obj = torch.tensor(train_obj, dtype=torch.float32)
print(train_obj)

tensor([[0.0050, 0.5000],
        [0.0050, 0.5000],
        [0.0050, 0.5000],
        [0.0050, 0.5000],
        [0.0050, 1.0000],
        [0.0050, 1.0000],
        [0.0050, 1.0000],
        [0.0050, 1.0000],
        [0.0050, 2.0000],
        [0.0050, 2.0000],
        [0.0050, 2.0000],
        [0.0050, 2.0000],
        [0.0406, 0.5000],
        [0.0406, 0.5000],
        [0.0406, 0.5000],
        [0.0406, 0.5000],
        [0.0406, 1.0000],
        [0.0406, 1.0000],
        [0.0406, 1.0000],
        [0.0406, 1.0000],
        [0.0406, 2.0000],
        [0.0406, 2.0000],
        [0.0406, 2.0000],
        [0.0406, 2.0000]])
tensor([[ 0.0450,  0.0500,  0.0090,  0.0190,  0.1520,  0.0010,  0.0210,  0.0270,
         -0.1990],
        [ 0.0390,  0.0540,  0.0130,  0.0640,  0.1800,  0.0010,  0.0260,  0.0210,
         -0.1990],
        [ 0.0470,  0.0660,  0.0140,  0.0280,  0.2280,  0.0010,  0.0330,  0.0400,
         -0.1990],
        [ 0.0430,  0.0570,  0.0120,  0.0370,  0.1870,  0.0010,  0.0270,  0.

In [6]:
from botorch import fit_gpytorch_model
from botorch.models.gp_regression import FixedNoiseGP
from botorch.models.transforms.outcome import Standardize
from botorch.models.model_list_gp_regression import ModelListGP
from gpytorch.mlls.sum_marginal_log_likelihood import SumMarginalLogLikelihood
from botorch.utils.transforms import unnormalize, normalize
from botorch.utils.multi_objective.box_decompositions.non_dominated import FastNondominatedPartitioning
from botorch.acquisition.multi_objective.monte_carlo import qExpectedHypervolumeImprovement
from botorch.sampling.samplers import SobolQMCNormalSampler
from botorch.optim.optimize import optimize_acqf

NOISE_SE = torch.tensor([1,1,1,1,1,1,1,1,1])
#print(NOISE_SE)

# define models for objective and constraint
models = []
for i in range(train_obj.shape[-1]):
    train_y = train_obj[..., i:i+1]
    train_yvar = torch.full_like(train_y, NOISE_SE[i] ** 2)
    models.append(FixedNoiseGP(train_x, train_y, train_yvar, outcome_transform=Standardize(m=1)))
model = ModelListGP(*models)
mll = SumMarginalLogLikelihood(model.likelihood, model)
#fit_gpytorch_model(mll)

BATCH_SIZE = 5
NUM_RESTARTS = 2
RAW_SAMPLES = 4
MC_SAMPLES = 16

standard_bounds = torch.tensor([[0.001, 0.1],[0.05, 3]])
#print(standard_bounds)
ref_point=torch.tensor([0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,-1])

with torch.no_grad():
  #pred = model.posterior(normalize(train_x, standard_bounds)).mean
  pred = model.posterior(train_x).mean
partitioning = FastNondominatedPartitioning(ref_point, Y=pred)
sampler = SobolQMCNormalSampler(num_samples=MC_SAMPLES)
acq_func = qExpectedHypervolumeImprovement(model,ref_point,partitioning,sampler)

candidates, _ = optimize_acqf(
        acq_function=acq_func,
        bounds=standard_bounds,
        q=BATCH_SIZE,
        num_restarts=NUM_RESTARTS,
        raw_samples=RAW_SAMPLES,  # used for intialization heuristic
        #options={"batch_limit": 5, "maxiter": 200},
        sequential=True)
#candidates =  unnormalize(candidates.detach(), bounds=standard_bounds)
print(candidates)

tensor([[0.0407, 0.6501],
        [0.0210, 2.9195],
        [0.0253, 1.1993],
        [0.0382, 2.7602],
        [0.0306, 1.8583]])


In [7]:
BATCH_SIZE = 6
NUM_RESTARTS = 2
RAW_SAMPLES = 4
MC_SAMPLES = 16

standard_bounds = torch.tensor([[0.001, 0.1],[0.05, 3]])
#print(standard_bounds)
ref_point=torch.tensor([0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,-1])

with torch.no_grad():
  #pred = model.posterior(normalize(train_x, standard_bounds)).mean
  pred = model.posterior(train_x).mean
partitioning = FastNondominatedPartitioning(ref_point, Y=pred)
sampler = SobolQMCNormalSampler(num_samples=MC_SAMPLES)
acq_func = qExpectedHypervolumeImprovement(model,ref_point,partitioning,sampler)

candidates, _ = optimize_acqf(
        acq_function=acq_func,
        bounds=standard_bounds,
        q=BATCH_SIZE,
        num_restarts=NUM_RESTARTS,
        raw_samples=RAW_SAMPLES,  # used for intialization heuristic
        #options={"batch_limit": 5, "maxiter": 200},
        sequential=True)
#candidates =  unnormalize(candidates.detach(), bounds=standard_bounds)
print(candidates)

tensor([[0.0410, 0.5438],
        [0.0157, 2.6049],
        [0.0225, 2.1782],
        [0.0453, 2.8812],
        [0.0182, 1.0176],
        [0.0457, 1.9228]])


In [8]:
BATCH_SIZE = 7
NUM_RESTARTS = 2
RAW_SAMPLES = 4
MC_SAMPLES = 16

standard_bounds = torch.tensor([[0.001, 0.1],[0.05, 3]])
#print(standard_bounds)
ref_point=torch.tensor([0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,-1])

with torch.no_grad():
  #pred = model.posterior(normalize(train_x, standard_bounds)).mean
  pred = model.posterior(train_x).mean
partitioning = FastNondominatedPartitioning(ref_point, Y=pred)
sampler = SobolQMCNormalSampler(num_samples=MC_SAMPLES)
acq_func = qExpectedHypervolumeImprovement(model,ref_point,partitioning,sampler)

candidates, _ = optimize_acqf(
        acq_function=acq_func,
        bounds=standard_bounds,
        q=BATCH_SIZE,
        num_restarts=NUM_RESTARTS,
        raw_samples=RAW_SAMPLES,  # used for intialization heuristic
        #options={"batch_limit": 5, "maxiter": 200},
        sequential=True)
#candidates =  unnormalize(candidates.detach(), bounds=standard_bounds)
print(candidates)

tensor([[0.0445, 0.6660],
        [0.0421, 2.7245],
        [0.0381, 1.8420],
        [0.0399, 0.1174],
        [0.0369, 1.4341],
        [0.0096, 0.3995],
        [0.0301, 0.8591]])


In [9]:
BATCH_SIZE = 8
NUM_RESTARTS = 2
RAW_SAMPLES = 4
MC_SAMPLES = 16

standard_bounds = torch.tensor([[0.001, 0.1],[0.05, 3]])
#print(standard_bounds)
ref_point=torch.tensor([0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,-1])

with torch.no_grad():
  #pred = model.posterior(normalize(train_x, standard_bounds)).mean
  pred = model.posterior(train_x).mean
partitioning = FastNondominatedPartitioning(ref_point, Y=pred)
sampler = SobolQMCNormalSampler(num_samples=MC_SAMPLES)
acq_func = qExpectedHypervolumeImprovement(model,ref_point,partitioning,sampler)

candidates, _ = optimize_acqf(
        acq_function=acq_func,
        bounds=standard_bounds,
        q=BATCH_SIZE,
        num_restarts=NUM_RESTARTS,
        raw_samples=RAW_SAMPLES,  # used for intialization heuristic
        #options={"batch_limit": 5, "maxiter": 200},
        sequential=True)
#candidates =  unnormalize(candidates.detach(), bounds=standard_bounds)
print(candidates)

tensor([[0.0206, 0.1915],
        [0.0267, 2.3496],
        [0.0162, 1.2605],
        [0.0317, 2.0304],
        [0.0199, 2.9711],
        [0.0141, 0.6094],
        [0.0364, 2.2300],
        [0.0199, 2.5874]])


In [10]:
BATCH_SIZE = 9
NUM_RESTARTS = 2
RAW_SAMPLES = 4
MC_SAMPLES = 16

standard_bounds = torch.tensor([[0.001, 0.1],[0.05, 3]])
#print(standard_bounds)
ref_point=torch.tensor([0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,-1])

with torch.no_grad():
  #pred = model.posterior(normalize(train_x, standard_bounds)).mean
  pred = model.posterior(train_x).mean
partitioning = FastNondominatedPartitioning(ref_point, Y=pred)
sampler = SobolQMCNormalSampler(num_samples=MC_SAMPLES)
acq_func = qExpectedHypervolumeImprovement(model,ref_point,partitioning,sampler)

candidates, _ = optimize_acqf(
        acq_function=acq_func,
        bounds=standard_bounds,
        q=BATCH_SIZE,
        num_restarts=NUM_RESTARTS,
        raw_samples=RAW_SAMPLES,  # used for intialization heuristic
        #options={"batch_limit": 5, "maxiter": 200},
        sequential=True)
#candidates =  unnormalize(candidates.detach(), bounds=standard_bounds)
print(candidates)

tensor([[2.3095e-02, 6.9522e-01],
        [4.1864e-02, 2.8981e+00],
        [1.9530e-02, 1.8174e-01],
        [2.3179e-03, 2.2351e+00],
        [1.7198e-03, 1.5639e+00],
        [3.5096e-02, 1.3591e+00],
        [3.4290e-02, 2.1119e+00],
        [4.0977e-02, 2.5181e+00],
        [2.8614e-02, 2.9633e+00]])


In [11]:
BATCH_SIZE = 10
NUM_RESTARTS = 2
RAW_SAMPLES = 4
MC_SAMPLES = 16

standard_bounds = torch.tensor([[0.001, 0.1],[0.05, 3]])
#print(standard_bounds)
ref_point=torch.tensor([0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,0.0001,-1])

with torch.no_grad():
  #pred = model.posterior(normalize(train_x, standard_bounds)).mean
  pred = model.posterior(train_x).mean
partitioning = FastNondominatedPartitioning(ref_point, Y=pred)
sampler = SobolQMCNormalSampler(num_samples=MC_SAMPLES)
acq_func = qExpectedHypervolumeImprovement(model,ref_point,partitioning,sampler)

candidates, _ = optimize_acqf(
        acq_function=acq_func,
        bounds=standard_bounds,
        q=BATCH_SIZE,
        num_restarts=NUM_RESTARTS,
        raw_samples=RAW_SAMPLES,  # used for intialization heuristic
        #options={"batch_limit": 5, "maxiter": 200},
        sequential=True)
#candidates =  unnormalize(candidates.detach(), bounds=standard_bounds)
print(candidates)

tensor([[0.0470, 0.4857],
        [0.0385, 2.6448],
        [0.0457, 0.9128],
        [0.0121, 0.6568],
        [0.0368, 1.5865],
        [0.0472, 0.1821],
        [0.0228, 1.5860],
        [0.0385, 1.8234],
        [0.0451, 0.6220],
        [0.0494, 2.2529]])
